# CollegeScorecard

In [5]:
import pandas as pd
df = pd.read_csv('../data/raw/College_Scorecard_Raw_Data_05192025/MERGED2023_24_PP.csv', low_memory=False)
df

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,COUNT_WNE_MALE1_P11,GT_THRESHOLD_P11,MD_EARN_WNE_INC1_P11,MD_EARN_WNE_INC2_P11,MD_EARN_WNE_INC3_P11,MD_EARN_WNE_INDEP0_P11,MD_EARN_WNE_INDEP1_P11,MD_EARN_WNE_MALE0_P11,MD_EARN_WNE_MALE1_P11,SCORECARD_SECTOR
0,100654,100200.0,1002.0,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,www.aamu.edu/admissions-aid/tuition-fees/net-p...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1,100663,105200.0,1052.0,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,https://www.uab.edu/,https://tcc.ruffalonl.com/University of Alabam...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2,100690,2503400.0,25034.0,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,https://www.amridgeuniversity.edu/,https://www2.amridgeuniversity.edu:9091/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
3,100706,105500.0,1055.0,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu/,finaid.uah.edu/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,100724,100500.0,1005.0,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu/,www.alasu.edu/cost-aid/tuition-costs/net-price...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6424,49382201,4283601.0,42836.0,College Unbound - Newport/Aquidneck Island,Newport,RI,028400000,New England Commission on Higher Education,https://www.collegeunbound.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
6425,49425001,2609404.0,26094.0,Valley College - Fairlawn - School of Nursing,Fairlawn,OH,443333631,Accrediting Commission of Career Schools and C...,https://www.valley.edu/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
6426,49501301,4247201.0,42472.0,Western Maricopa Education Center - Southwest ...,Buckeye,AZ,85326-5705,Council on Occupational Education,https://west-mec.edu/findyourhappy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
6427,49501302,4247202.0,42472.0,Western Maricopa Education Center - Northeast ...,Phoenix,AZ,85027-0000,Council on Occupational Education,https://west-mec.edu/findyourhappy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13


Not much needs to be done, just updating dtypes

In [6]:
import yaml

# Load YAML
with open("../data/raw/College_Scorecard_Raw_Data_05192025/data.yaml", "r") as f:
    y = yaml.safe_load(f)

# Build source → dtype map
dtype_map = {}
for key, info in y["dictionary"].items():
    src = info.get("source")
    typ = info.get("type")

    if src and typ:
        if typ == "integer":
            dtype = "Int64"
        elif typ == "float":
            dtype = "float"
        else:
            dtype = "string"
        dtype_map[src] = dtype

for override_col in ["OPEID", "OPEID6"]:
    if override_col in df.columns:
        dtype_map[override_col] = "Int64"

# Apply dtypes only if column exists
for col, dtype in dtype_map.items():
    if col in df.columns:
        try:
            df[col] = df[col].astype(dtype)
        except Exception as e:
            print(f"Could not convert {col} to {dtype}: {e}")

Could not convert C150_L4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert C150_4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert C200_L4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert C200_4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert RET_FT4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert RET_FTL4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert RET_PT4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert RET_PTL4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert TRANS_4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert TRANS_L4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert C100_4_POOLED_SUPP to float: could not convert string to float: 'PS'
Could not convert C100_L4_POOLED

## Exporting Data

In [7]:
all_years = []
for yr in range(4, 24):
    origin_file = f'MERGED20{yr:02d}_{yr+1:02d}_PP.csv'
    # destination_file = f'CollegeScorecard_{yr+2000}.csv'
    destination_file = f'CollegeScorecard_{yr+2000}.pkl'

    df = pd.read_csv(f'../data/raw/College_Scorecard_Raw_Data_05192025/{origin_file}', low_memory=False)

    for override_col in ["OPEID", "OPEID6"]:
        if override_col in df.columns:
            dtype_map[override_col] = "Int64"

    for col, dtype in dtype_map.items():
        if col in df.columns:
            try:
                df[col] = df[col].astype(dtype)
            except Exception as e:
                pass

    df = df.assign(Year=yr + 2000)
    cols = ["Year"] + [c for c in df.columns if c != "Year"]
    df = df[cols]

    # df.to_csv(f'../data/processed/{destination_file}', index=False)
    df.to_pickle(f'../data/processed/{destination_file}')
    all_years.append(df)
    print(origin_file, destination_file)

merged_df = pd.concat(all_years, ignore_index=True)
# merged_df.to_csv(f'../data/processed/CollegeScorecard_AllYears.csv', index=False)
merged_df.to_pickle(f'../data/processed/CollegeScorecard_AllYears.pkl')

MERGED2004_05_PP.csv CollegeScorecard_2004.pkl
MERGED2005_06_PP.csv CollegeScorecard_2005.pkl
MERGED2006_07_PP.csv CollegeScorecard_2006.pkl
MERGED2007_08_PP.csv CollegeScorecard_2007.pkl
MERGED2008_09_PP.csv CollegeScorecard_2008.pkl
MERGED2009_10_PP.csv CollegeScorecard_2009.pkl
MERGED2010_11_PP.csv CollegeScorecard_2010.pkl
MERGED2011_12_PP.csv CollegeScorecard_2011.pkl
MERGED2012_13_PP.csv CollegeScorecard_2012.pkl
MERGED2013_14_PP.csv CollegeScorecard_2013.pkl
MERGED2014_15_PP.csv CollegeScorecard_2014.pkl
MERGED2015_16_PP.csv CollegeScorecard_2015.pkl
MERGED2016_17_PP.csv CollegeScorecard_2016.pkl
MERGED2017_18_PP.csv CollegeScorecard_2017.pkl
MERGED2018_19_PP.csv CollegeScorecard_2018.pkl
MERGED2019_20_PP.csv CollegeScorecard_2019.pkl
MERGED2020_21_PP.csv CollegeScorecard_2020.pkl
MERGED2021_22_PP.csv CollegeScorecard_2021.pkl
MERGED2022_23_PP.csv CollegeScorecard_2022.pkl
MERGED2023_24_PP.csv CollegeScorecard_2023.pkl
MERGED2004_05_PP.csv CollegeScorecard_2004.pkl
MERGED2005_06